In [1]:
from collections import OrderedDict
import torch
from torchsummary import summary
import torch.nn as nn
import numpy as np
from torch.nn import functional as F
from collections import OrderedDict

In [2]:
import os

import torch
import numpy as np
import scipy.io as scio
from torch.utils.data import Dataset, DataLoader
import random

trainfolder='/datadisk/sdc/wangruibo/maml_e2e/'
testfolder='/datadisk/sdc/wangruibo/maml_e2e/'

class dataset(Dataset):

    def __init__(self,input_path, target_path, mode = 'train', seedNN = 0):
        super(dataset, self).__init__()
        self.input_data = []
        self.target_data = []
        input_data_meta = np.load(input_path)
        target_data_meta = np.load(target_path)
        self.length = input_data_meta.shape[0]
        
#         random.seed(seedNN)
#         ind_shuffle = random.sample(range(0, self.length), self.length)
        # random seed inside
        if mode == 'train':
            self.target_data = torch.randint(0, 2, (10000,72*14, 4))
            self.input_data = input_data_meta
#             self.input_data = input_data_meta[:int(0.2 * self.length)]
        else:
            self.target_data = torch.randint(0, 2, (10000,72*14, 4))
#             self.input_data = input_data_meta[int(0.2 * self.length):]
            self.input_data = input_data_meta
        self.length = self.input_data.shape[0]
    def __len__(self):
        return self.length

    def __getitem__(self, index):
#         print(self.input_data[index].shape)
        input_data = torch.from_numpy(self.input_data[index])
        target_data = self.target_data[index]
        return input_data, target_data

class UmaDatasets(Dataset):
    def __init__(self,trainFolder=trainfolder,testFolder=testfolder):
       # number of tasks
        self.length=0
        self.traindataloader_Loc={}
        self.testdataloader_Loc={}
#         self.Channel_idx=['140-200-low', '140-200-medium', '140-200-high'
#                             , '20-80-low','20-80-medium','20-80-high',
#                            '80-140-low','80-140-medium','80-140-high']
        self.Channel_idx=['140-200-low', '140-200-medium', '140-200-high']
#         self.Channel_idx=['20-80-low','20-80-medium','20-80-high',
#                            '80-140-low','80-140-medium','80-140-high']
        self.strength_range= ['0.0','0.66','1.33','2.0']  
        self.TrainDataFolder=trainFolder
        self.TestDataFolder=testFolder
        self.load()
        
    def load(self):
#         print('Train Data Folder: ',self.TrainDataFolder)
        for il in self.Channel_idx:
            if il.split('-')[2] == 'low':
                m,n = self.strength_range[0],self.strength_range[1]
            elif il.split('-')[2] == 'medium':
                m,n = self.strength_range[1],self.strength_range[2]
            else:
                m,n = self.strength_range[2],self.strength_range[3]
#             print(il)
            input_path11 = self.TrainDataFolder + il + '/dataset_UMa_D={}-{}_&_A={}-{}_s=10000.npy'.format(float(il.split('-')[0]), float(il.split('-')[1]),m,n)
            target_path1 = self.TrainDataFolder + 'bit_labels_s_size=10000.npy'
#             print(il,' ',len(input_path11))
#             print(len(target_path1))
            trainset= dataset(input_path11, target_path1, mode='train', seedNN = 0)
            
            self.length+=trainset.length
            self.traindataloader_Loc[il]= trainset
            
#             self.TestDataFolder
            input_path21 = self.TrainDataFolder + il + '/dataset_UMa_D={}-{}_&_A={}-{}_s=10000.npy'.format(float(il.split('-')[0]), float(il.split('-')[1]),m,n)
            target_path2 = self.TrainDataFolder + 'bit_labels_s_size=10000.npy'
            testset    = dataset(input_path21, target_path2, mode='test', seedNN = 0)
            self.testdataloader_Loc[il]= testset
            
    def sample_task(self, mode='train'):
        return np.random.choice(self.Channel_idx)
    def sample_datas(self, name, K = 10, mode='train'):
        data=[]
        label=[]
        if mode == 'train':
            tidx=np.random.choice(range(len(self.traindataloader_Loc[name])),K)
            for i in tidx:
                data.append(self.traindataloader_Loc[name][i][0])
                label.append(self.traindataloader_Loc[name][i][1])
        else:
            tidx=np.random.choice(range(len(self.testdataloader_Loc[name][0])),K)
#             datas=self.testdataloader_Loc[name][0]
#             labels=self.testdataloader_Loc[name][1]
            for i in tidx:
                data.append(self.testdataloader_Loc[name][i][0])
                label.append(self.testdataloader_Loc[name][i][1])
        return data,label
    
    def __len__(self):
        return self.length

In [3]:
uma_dataset = UmaDatasets(trainfolder,testfolder)

In [4]:
from __future__ import print_function
import torch.nn as nn
import torch
from torch.nn import functional as F


def gen_labeling(bit_per_cu):
    
        def genCode(n):
                if n == 0:
                    return ['']

                code1 = genCode(n-1)
                code2 = []
                for codeWord in code1:
                    code2 = [codeWord] + code2

                for i in range(len(code1)):
                    code1[i] += '0'
                for i in range(len(code2)):
                    code2[i] += '1'
                return code1 + code2  

        k = bit_per_cu // 2 # bits per dimension
        syms = 2**k
        m = syms**2
        # gray labeling
        gray_labeling = []
        for i in range(2**k):
            gray_labeling.append(np.array([int(e) for e in  genCode(k)[i]]))
        gray_labeling = np.array(gray_labeling)

        encoding_table_256_qam = gray_labeling[:,::-1]

        encoding_table_256_qam = np.hstack( [np.kron(encoding_table_256_qam,np.ones((syms,1))), np.tile(encoding_table_256_qam,(syms,1)) ])

        natural_number = np.sum(2**np.arange(2*k)[::-1] * encoding_table_256_qam,axis=1)
        table_gray = np.zeros((m,m))
        for bvec in range(m):
            table_gray[bvec,np.where(natural_number==bvec)[0]] =1

        return table_gray

def gen_const(bit_per_cu):
    
    points_per_dim = 2**(bit_per_cu // 2)
    
    A = np.arange(0, points_per_dim)
    A = A - np.mean(A)
    
    C = []
    for i in A:
        for j in A:
            C.append([i,j])
    C = np.array(C)
    
    L = gen_labeling(bit_per_cu)
    L = np.argmax(L, axis=1)
    C = C[L]
    
    return C


class GS(nn.Module):
    def __init__(self,  bit_per_cu, init_qam=False):
        super(GS, self).__init__()
        self.bit_per_cu = bit_per_cu
        self.init_qam = init_qam
        if self.init_qam:
            C = gen_const(self.bit_per_cu)
            en_point = np.mean(np.sum(np.square(C), axis=1))
            C = C / np.sqrt(en_point)
            self.Cr = torch.tensor(C[:,0],dtype = torch.float())#,requires_grad=True)
            self.Ci = torch.tensor(C[:,1],dtype = torch.float())#,requires_grad=True)
            self.Cr = nn.Parameter(self.Cr)
            self.Ci = nn.Parameter(self.Ci)
        else:
            const_size = 2**self.bit_per_cu
            self.Cr = torch.normal(mean = 0.0,std=0.05,size = (1,const_size)).squeeze()#.requires_grad_(True)
            self.Ci = torch.normal(mean = 0.0,std=0.05,size = (1,const_size)).squeeze()#.requires_grad_(True)
            self.Cr = nn.Parameter(self.Cr)
            self.Ci = nn.Parameter(self.Ci)
    def get_const(self):
        C = torch.complex(self.Cr, self.Ci)
        # Center the constellation
        C = C - torch.mean(C)
        
        # Normalization
        en_point = torch.mean(torch.square(torch.real(C)) + torch.square(torch.imag(C)))
        norm_fac = torch.complex(torch.sqrt(en_point), torch.tensor(0.0).cuda())
        C = torch.div(C, norm_fac)
        if np.inf in C:
            C[C == np.inf] = torch.tensor(0.0).cuda()
        
        return C
    def get_const_para(self,weight):
        C = torch.complex(weight[0], weight[1])
        # Center the constellation
        C = C - torch.mean(C)
        
        # Normalization
        en_point = torch.mean(torch.square(torch.real(C)) + torch.square(torch.imag(C)))
        norm_fac = torch.complex(torch.sqrt(en_point), torch.tensor(0.0).cuda())
        C = torch.div(C, norm_fac)
        if np.inf in C:
            C[C == np.inf] = torch.tensor(0.0).cuda()
        
        return C
    def forward(self,s):
        
        batch_size = s.shape[0]
        C = self.get_const()
        C = torch.tile(torch.unsqueeze(C, axis=0), (batch_size, 1))
                
        # Mapping
        s = s.long()
        x = torch.gather(C,1,s)
        
        return x
    def parameter(self,s,weight):
        batch_size = s.shape[0]
        C = self.get_const_para(weight)
        C = torch.tile(torch.unsqueeze(C, axis=0), (batch_size, 1))
                
        # Mapping
        s = s.long()
        x = torch.gather(C,1,s)
        
        return x
# class SeparableConv2d(torch.nn.Module):
#     def __init__(self, 
#                  in_channels,
#                  out_channels,
#                  kernel_size,
#                  dilation_size,
#                  padding_size,
#                  bias=True
#         ):
#         super(SeparableConv2d,self).__init__()
#         self.spatialConv = convnxn(
#              in_channels=in_channels,
#              out_channels=out_channels,
#              kernel_size=kernel_size,
#              padding_size=padding_size,
#              dilation_size=dilation_size,
#              groups_size=in_channels
#         )
#         self.pointConv = convnxn(
#              in_channels=in_channels,
#              out_channels=out_channels,
#              kernel_size=1,
#              padding_size=0,
#              dilation_size=1,
#              groups_size = 1
#         )
    
#     def forward(self, x, params=None, episode=None):
#         return self.pointConv(self.spatialConv(x, get_child_dict(params, 'spatialConv')), get_child_dict(params, 'pointConv'))


class ResNetBlock(nn.Module):
    
    def __init__(self, filters, kernel_size,dilation_rate, use_dense,padding_size,training):
        super(ResNetBlock, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.dilation_rate = dilation_rate
        self.use_dense = use_dense
        self.padding_size = padding_size
        self.training = training
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        if self.use_dense:
            self.dense = nn.Linear(self.filters,self.filters,bias = False)
        self.bn1 = nn.BatchNorm2d(self.filters, affine = self.training)
        self.conv1 = nn.Conv2d(self.filters,self.filters, self.kernel_size, dilation=self.dilation_rate, padding=self.padding_size,groups = 1)
        self.bn2 = nn.BatchNorm2d(self.filters, affine = self.training)
        self.conv2 = nn.Conv2d(self.filters,self.filters, self.kernel_size, dilation=self.dilation_rate, padding=self.padding_size,groups = 1)
    
    def forward(self, x):
        z = x
        
        z = self.bn1(z)
        z = self.relu1(z)
        z = self.conv1(z)
        z = self.bn2(z)
        z = self.relu2(z)
        z = self.conv2(z)

        if self.use_dense:
            y = self.dense(x)
        else:
            y = x
        z = z + y
        
        return z
    def parameter(self,x,para,weight):
        z = x
        z = F.batch_norm(z,running_var = weight[1],running_mean = weight[0],weight = para[0],bias = para[1],training = self.training)
        z = F.relu(z)
        z = F.conv2d(z,para[2],para[3],dilation=self.dilation_rate, padding=self.padding_size)
        z = F.batch_norm(z,running_var = weight[4],running_mean = weight[3],weight = para[4],bias = para[5],training = self.training)
        z = F.relu(z)
        z = F.conv2d(z,para[6],para[7],dilation=self.dilation_rate, padding=self.padding_size)
        if self.use_dense:
            y = self.dense(x)
        else:
            y = x
        z = z + y
        return z
class OFDM:
    
    # num_subc: number of subcarriers
    # cp_length: Cycle prefix duration [symbols]
    # use_cp: If True, uses a cycling-prefix, otherwise does not.
    def __init__(self, num_subc, cp_length, use_cp=True):
        
        self.num_subc = num_subc
        self.cp_length = cp_length
        self.use_cp = use_cp
    
    # x_f: Baseband symbols in the frequency domain [batch size, number of subcarriers, number of OFDM symbols]
    # The ordering of the subcarriers is assumed to be as follows: negatve frequencies (in order of decreasingly negative frequency),
    #  then center frequency, then positive frequencies.
    def modulate(self, x_f):
        
        # Checking the number of subcarriers is large enough given the CP length
        # tf.debugging.assert_positive(tf.shape(x_f)[1] - self.cp_length, "Number of subcarriers too small given the specified CP length")
        
        x_f = torch.transpose(x_f, 2, 1)
        
        # FFT shift
        x_f = torch.fft.ifftshift(x_f, dim=2)
        
        # Moving to time domain
        x = torch.fft.ifft(x_f)*torch.complex(torch.sqrt(torch.tensor(x_f.shape[-1])), torch.tensor(0.0))
        
        # Adding CP
        if self.use_cp:
            x = torch.cat((x[:,:,-self.cp_length:], x), dim=2)
        
        # Reshaping to get a single time domain signal
        x = x.view(x.shape[0], -1)
        
        return x
        
    
    # y_t: Baseband symbols in the time domain [batch size, time steps]
    def demodulate(self, y_t):
        
        if self.use_cp:
            symbol_length = self.num_subc + self.cp_length
        else:
            symbol_length = self.num_subc
        
        # Checking the signal dimension matches the specified CP length and number of subcarriers
        num_symbs = torch.div(y_t.shape[1], symbol_length)
        rem_symbs = torch.fmod(torch.tensor(y_t.shape[1]), symbol_length)
#         tf.debugging.assert_positive(num_symbs, "Signal duration is too short given specified OFDM configuration")
#         tf.debugging.assert_equal(rem_symbs, 0, "Signal duration does not match specified OFDM configuration")
        
        # Reshaping
        y = y_t.view(y_t.shape[0], -1, symbol_length)
        
        # Removing CP
        if self.use_cp:
            y = y[:,:,self.cp_length:]
        
        # Moving to frequency domain
        y = torch.fft.fft(y)/torch.complex(torch.sqrt(torch.tensor(y.shape[-1])), torch.tensor(0.0))
        
        # FFT shift
        y = torch.fft.fftshift(y, dim=2)
        
        y = torch.transpose(y, 2, 1)
        
        return y
    

class MultiPathChannel(nn.Module):
    
    def __init__(self):
        super(MultiPathChannel, self).__init__()
         
    # x: time domain baseband channel input symbols, with shape [batch_size, number of time steps]
    # h: channel response in time domain, with shape [batch_size, number of time steps, number of taps]
    # noise_power_db: Noise power in dB. Scalar
    def forward(self, x, h, noise_power_db):
        
        batch_size = x.shape[0]
        time_steps_no = x.shape[1]
        taps_no = h.shape[2]
        
        # If extra time steps are available for the channel, we just ignore them
        h = h[:,:time_steps_no,:]
        # Reshaping the channel input symbols
        x = torch.unsqueeze(x, dim=2)
        x = torch.tile(x, (1, 1, taps_no))
        padding_r = torch.zeros((batch_size, taps_no, taps_no))
        padding_i = torch.zeros((batch_size, taps_no, taps_no))
        padding = torch.complex(padding_r,padding_i)
        padding = padding.cuda()
        x = torch.cat((x, padding), axis=1)
        x_ = torch.transpose(torch.zeros_like(torch.transpose(torch.zeros_like(x), 0,2)), 1,2)
        for i in range(taps_no):
            z = x[:,:,i]
            z = torch.roll(z, i, 1)
            x_[i] = z
        x_ = torch.transpose(x_, 0,1)
        x_ = torch.transpose(x_, 1,2)
        x = x_[:,:time_steps_no,:] # [batch size, number of timer steps, number of taps]
        # Channel transfer function
        y = torch.sum(h*x, axis=2)
        noise_std = torch.sqrt(0.5*(torch.pow(10.0, noise_power_db/10.0)))
        noise_std = torch.unsqueeze(noise_std, dim=1)
        noise_std = noise_std.numpy()
        noise = (np.random.normal(0.0, noise_std,size = (y.shape[0],y.shape[1])), np.random.normal(0.0, noise_std,size = (y.shape[0],y.shape[1])))
        noise = torch.complex(torch.tensor(noise[0]), torch.tensor(noise[1]))
        noise = noise.cuda()
        y = y + noise
        y = y.to(dtype=torch.complex64)
        return y

class RXNN(nn.Module):
    
    def __init__(self, bit_per_cu,training):
        super(RXNN, self).__init__()
        
        self.bit_per_cu = bit_per_cu
        self.training = training
        self.conv_in = nn.Conv2d(5,256, (3,3), dilation=(1,1), padding=(1,1))#padding 1,1
        self.res1 = ResNetBlock(256, (3,3), (3,1),False,(3,1),training)#padding 3,1
        self.res2 = ResNetBlock(256, (3,3), (6,2), False,(6,2),training)#padding 6,2
        self.res3 = ResNetBlock(256, (3,3), (6,2), False,(6,2),training)#padding 6,2
        self.res4 = ResNetBlock(256, (3,3), (3,1), False,(3,1),training)#padding 3,1
        self.conv_out = nn.Conv2d(256,4, (1,1), dilation=(1,1), padding=(0,0))#padding 0,0
    
    # all inputs of shape [nm subframes, num subcarriers, block length]
    # y: received signal
    # snr_db: SNR [dB]
    def forward(self, y, snr_db):
        
        # Positional encoding
        num_frames = y.shape[0]
        num_subc = y.shape[1]
        block_length = y.shape[2]
        spectral_pos_enc = torch.linspace(0.0, 1.0, num_subc)
        time_pos_enc = torch.linspace(0.0, 1.0, block_length)
        spectral_pos_enc = torch.unsqueeze(spectral_pos_enc, dim =1)
        spectral_pos_enc = torch.tile(spectral_pos_enc, (1, block_length))
        time_pos_enc = torch.unsqueeze(time_pos_enc, axis=0)
        time_pos_enc = torch.tile(time_pos_enc, (num_subc, 1))
        pos_enc = torch.stack([spectral_pos_enc, time_pos_enc], dim =2)
        pos_enc = torch.unsqueeze(pos_enc, axis=0)
        pos_enc = torch.tile(pos_enc, (num_frames, 1, 1, 1))

        #         # Inputs
        snr_db = snr_db.cuda()
        pos_enc = pos_enc.cuda()
        z = torch.stack((torch.real(y), torch.imag(y),snr_db),dim = 3)
        z = torch.cat((z, pos_enc), dim=3)
        z = torch.transpose(z, 1, 3)
        z = torch.transpose(z, 2, 3)
        # Call conv layers
#         print(z)
        z = z.float()
        z = self.conv_in(z)
        z = self.res1(z)
        z = self.res2(z)
        z = self.res3(z)
        z = self.res4(z)
        z = self.conv_out(z)
        z = torch.transpose(z, 1, 3)
        z = torch.transpose(z, 1, 2)
        return z
    def parameter(self, y, snr_db,weight,temp_weight):
        num_frames = y.shape[0]
        num_subc = y.shape[1]
        block_length = y.shape[2]
        spectral_pos_enc = torch.linspace(0.0, 1.0, num_subc)
        time_pos_enc = torch.linspace(0.0, 1.0, block_length)
        spectral_pos_enc = torch.unsqueeze(spectral_pos_enc, dim =1)
        spectral_pos_enc = torch.tile(spectral_pos_enc, (1, block_length))
        time_pos_enc = torch.unsqueeze(time_pos_enc, axis=0)
        time_pos_enc = torch.tile(time_pos_enc, (num_subc, 1))
        pos_enc = torch.stack([spectral_pos_enc, time_pos_enc], dim =2)
        pos_enc = torch.unsqueeze(pos_enc, axis=0)
        pos_enc = torch.tile(pos_enc, (num_frames, 1, 1, 1))

        #         # Inputs
        snr_db = snr_db.cuda()
        pos_enc = pos_enc.cuda()
        z = torch.stack((torch.real(y), torch.imag(y),snr_db),dim = 3)
        z = torch.cat((z, pos_enc), dim=3)
        z = torch.transpose(z, 1, 3)
        z = torch.transpose(z, 2, 3)
        # Call conv layers
#         print(z)
        z = z.float()
        z = F.conv2d(z,weight[2],weight[3],dilation=(1,1), padding=(1,1))
        z = self.res1.parameter(z,weight[4:12],temp_weight[6:9]+temp_weight[13:16])
        z = self.res2.parameter(z,weight[12:20],temp_weight[20:23]+temp_weight[27:30])
        z = self.res3.parameter(z,weight[20:28],temp_weight[34:37]+temp_weight[41:44])
        z = self.res4.parameter(z,weight[28:36],temp_weight[48:51]+temp_weight[55:58])
        z = F.conv2d(z,weight[36],weight[37], dilation=(1,1), padding=(0,0))
        z = torch.transpose(z, 1, 3)
        z = torch.transpose(z, 1, 2)
        return z
    
class meta_Net_DNN(nn.Module):
    def __init__(self): # it only gets paramters from other network's parameters
        super(meta_Net_DNN, self).__init__()
        self.gs = GS(4)
        self.ofdm = OFDM(72, 6, False)
        self.channel = MultiPathChannel()
        self.rx = RXNN(4,True)
    
    def forward(self,snr_db,h,s):
        noise_power_db = -snr_db
#         print(s.shape)
        x = self.gs(s)
        x = x.view(10,72,14)
        x = self.ofdm.modulate(x)
        y = self.channel(x,h,noise_power_db)
        y = self.ofdm.demodulate(y)
        snr_db_rx = torch.tile(torch.unsqueeze(torch.unsqueeze(snr_db, dim=1), dim=2), [1, 72, 14])
        y = self.rx(y,snr_db_rx)
        return y
    
    def parameter(self,snr_db,h,s,weight,temp_weight):
        noise_power_db = -snr_db
        x = self.gs.parameter(s,weight)
        x = x.view(10,72,14)
        x = self.ofdm.modulate(x)
        y = self.channel(x,h,noise_power_db)
        y = self.ofdm.demodulate(y)
        snr_db_rx = torch.tile(torch.unsqueeze(torch.unsqueeze(snr_db, dim=1), dim=2), [1, 72, 14])
        y = self.rx.parameter(y,snr_db_rx,weight,temp_weight)
        y = y.view(y.shape[0],y.shape[1]*y.shape[2],y.shape[3])
        return y

def meta_dnn(**kwargs):
    net = meta_Net_DNN(**kwargs)
    return net


In [12]:
net = meta_dnn()

In [13]:
import time
class MAML():
    def __init__(self, maml_model, tasks, inner_lr, meta_lr, K=10, inner_steps=1, tasks_per_meta_batch=10):
        
        # important objects
        self.tasks = tasks
        self.model = maml_model
        self.weight = []
        for k,v in self.model.state_dict().items():
            self.weight.append(v.clone())
        self.weights = list(maml_model.parameters())
#         self.weights = list(self.e2e_model.parameters()) # the maml weights we will be meta-optimising
#         self.criterion = nn.MSELoss()
        self.meta_optimiser = torch.optim.Adam(self.weights, meta_lr)
#         self.scheduler = torch.optim.lr_scheduler.StepLR(self.meta_optimiser, step_size=10, gamma=0.9)
        # hyperparameters
        self.inner_lr = inner_lr
        self.meta_lr = meta_lr
        self.K = K
        self.inner_steps = inner_steps # with the current design of MAML, >1 is unlikely to work well 
        self.tasks_per_meta_batch = tasks_per_meta_batch 
        # metrics
#         self.plot_every = 100
#         self.print_every = 500
        self.meta_losses = []
        self.ber_thing = []
    def inner_loop(self, task):
        # reset inner model to current maml weights
        temp_weights = [w.clone() for w in self.weights]
        temp_weight = [w.clone() for w in self.weight]
        # perform training on data sampled from task
        for step in range(self.inner_steps):
            X, y = self.tasks.sample_datas(task, self.K)
            X = torch.stack(X).float().cuda()
            y = torch.stack(y).float()
            snr_db = torch.DoubleTensor(1, 10).uniform_(0.0, 25.0).squeeze()
            A = np.flip([[[2**i for i in range(4)]]])
            s = torch.sum(y.cpu()*A, axis=2)
            s = s.cuda()
            y = y.cuda()
#             loss = F.binary_cross_entropy_with_logits(self.model.parameter(snr_db,X,s,temp_weights,temp_weight), y) / self.K
            loss = F.binary_cross_entropy_with_logits(self.model.parameter(snr_db,X,s,temp_weights,temp_weight), y)
            # compute grad and update inner loop weights
            grad = torch.autograd.grad(loss, temp_weights)
            temp_weights = [w - self.inner_lr * g for w, g in zip(temp_weights, grad)]
        # sample new data for meta-update and compute loss
        X, y = self.tasks.sample_datas(task, self.K,mode = 'test')
        X = torch.stack(X).float().cuda()
        y = torch.stack(y).float()
        snr_db = torch.DoubleTensor(1, 10).uniform_(0.0, 25.0).squeeze()
        A = np.flip([[[2**i for i in range(4)]]])
        s = torch.sum(y.cpu()*A, axis=2)
        s = s.cuda()
#         loss = self.criterion(self.model.parameterised(X,s, temp_weights), y) / self.K
        y = y.cuda()
#         loss = F.binary_cross_entropy_with_logits(self.model.parameter(snr_db,X,s,temp_weights,temp_weight), y) / self.K
        loss = F.binary_cross_entropy_with_logits(self.model.parameter(snr_db,X,s,temp_weights,temp_weight), y)
        return loss
    
    def main_loop(self, num_iterations):
        epoch_loss = 0
        start = 1
        for iteration in range(start, num_iterations+1):
            self.meta_optimiser.zero_grad()
            time_start = time.time()
            # compute meta loss
            meta_loss = 0
            for i in range(self.tasks_per_meta_batch):
                task = self.tasks.sample_task()
                meta_loss += self.inner_loop(task)
            # compute meta gradient of loss with respect to maml weights
            
            meta_grads = torch.autograd.grad(meta_loss, self.weights)
            
            # assign meta gradient to weights and take optimisation step
                                                                                                                                                                           
            for w, g in zip(self.weights, meta_grads):
                w.grad = g
            self.meta_optimiser.step()
#             self.scheduler.step()
            # log metrics
            epoch_loss += meta_loss.item() / self.tasks_per_meta_batch
            time_end = time.time()
            bertest = 0
            with torch.no_grad():
                X, y = self.tasks.sample_datas(task, self.K,mode = 'test')
                X = torch.stack(X).float().cuda()
                y = torch.stack(y).float()
                snr_db = torch.DoubleTensor(1, 10).uniform_(0.0, 25.0).squeeze()
                A = np.flip([[[2**i for i in range(4)]]])
                s = torch.sum(y.cpu()*A, axis=2)
                s = s.cuda()
    #         loss = self.criterion(self.model.parameterised(X,s, temp_weights), y) / self.K
                y = y.cuda()
                
                LLR = self.model(snr_db,X,s)
                LLR = torch.where(LLR >= 0, 1,-1)
                LLR = (LLR+1)/2
                LLR = LLR.view(LLR.shape[0],1008,4)
                bertest += (1-((LLR==y).sum().item()/(LLR.shape[0]*LLR.shape[1]*LLR.shape[2])))
            self.ber_thing.append(bertest)
            showStrings = '[Epoch: %3d/%3d][BER: %3.5f][CostTime: %3.5f s]' % (iteration,  num_iterations, bertest,time_end -time_start)
            print(showStrings, end='\n')
            file = open("/datadisk/sdc/wangruibo/Code_MAML_newframework_e2e/PyTorch-MAML-master/MAML_ORIGINIAL_5000/" + 'MAML' + '.txt' , 'w')
            file.write('BER test: ' + str(self.ber_thing) + "\n")
            file.close()
            if iteration % 500 == 0:
                print(self.model.state_dict()['gs.Cr'])
                torch.save(self.model.state_dict(), "/datadisk/sdc/wangruibo/Code_MAML_newframework_e2e/PyTorch-MAML-master/MAML_ORIGINIAL_5000/" + 'Checkpoints-Epk{}.pth'.format(iteration))

#             new_weight = OrderedDict()
#             for k,v in self.model.state_dict().items():
#                 new_weight[k] = v
#             index = 0
#             for k,v in new_weight.items():
#                 if 'running_mean' or 'running_var' or 'num_batches_tracked' not in k:
#                 #  or 'num_batches_tracked'
#                     new_weight[k] = self.weights[index]
#                     index += 1
#             self.temp_model.load_state_dict(new_weight)
#             torch.save(self.model.state_dict(), "/datadisk/sdc/wangruibo/new_maml/" + 'Checkpoints-Epk{}.pth'.format(iteration))
#             if iteration % self.print_every == 0:
#                 print("{}/{}. loss: {}".format(iteration, num_iterations, epoch_loss / self.plot_every))
#                 logger.info('Epoch:[{}/{}]\t loss={:.5f}'.format(iteration , num_iterations, epoch_loss / self.plot_every))
                
#             if iteration % self.plot_every == 0:
#                 self.meta_losses.append(epoch_loss / self.plot_every)
#                 epoch_loss = 0

In [14]:
maml_learning = MAML(net.cuda(),uma_dataset,inner_lr=0.00001, meta_lr=0.00001, K=10,inner_steps=50, tasks_per_meta_batch=1)

In [15]:
maml_learning.main_loop(10000)

[Epoch:   1/10000][BER: 0.50010][CostTime: 2.90619 s]
[Epoch:   2/10000][BER: 0.50751][CostTime: 2.86042 s]
[Epoch:   3/10000][BER: 0.49618][CostTime: 2.87395 s]
[Epoch:   4/10000][BER: 0.50308][CostTime: 2.89288 s]
[Epoch:   5/10000][BER: 0.50340][CostTime: 2.86844 s]
[Epoch:   6/10000][BER: 0.49814][CostTime: 2.85230 s]
[Epoch:   7/10000][BER: 0.51049][CostTime: 2.87181 s]
[Epoch:   8/10000][BER: 0.50114][CostTime: 2.90153 s]
[Epoch:   9/10000][BER: 0.49866][CostTime: 2.87824 s]
[Epoch:  10/10000][BER: 0.50020][CostTime: 2.85229 s]
[Epoch:  11/10000][BER: 0.50697][CostTime: 2.85151 s]
[Epoch:  12/10000][BER: 0.50461][CostTime: 2.89184 s]
[Epoch:  13/10000][BER: 0.49263][CostTime: 2.86769 s]
[Epoch:  14/10000][BER: 0.50893][CostTime: 2.86096 s]
[Epoch:  15/10000][BER: 0.50511][CostTime: 2.86122 s]
[Epoch:  16/10000][BER: 0.50397][CostTime: 2.84813 s]
[Epoch:  17/10000][BER: 0.49926][CostTime: 2.84399 s]
[Epoch:  18/10000][BER: 0.49462][CostTime: 2.85157 s]
[Epoch:  19/10000][BER: 0.48

KeyboardInterrupt: 

## FINETUNE ON NEW THING

In [9]:
model_dict = maml_learning.model.state_dict()

In [13]:
def gen_labeling(bit_per_cu):
    
        def genCode(n):
                if n == 0:
                    return ['']

                code1 = genCode(n-1)
                code2 = []
                for codeWord in code1:
                    code2 = [codeWord] + code2

                for i in range(len(code1)):
                    code1[i] += '0'
                for i in range(len(code2)):
                    code2[i] += '1'
                return code1 + code2  

        k = bit_per_cu // 2 # bits per dimension
        syms = 2**k
        m = syms**2
        # gray labeling
        gray_labeling = []
        for i in range(2**k):
            gray_labeling.append(np.array([int(e) for e in  genCode(k)[i]]))
        gray_labeling = np.array(gray_labeling)

        encoding_table_256_qam = gray_labeling[:,::-1]

        encoding_table_256_qam = np.hstack( [np.kron(encoding_table_256_qam,np.ones((syms,1))), np.tile(encoding_table_256_qam,(syms,1)) ])

        natural_number = np.sum(2**np.arange(2*k)[::-1] * encoding_table_256_qam,axis=1)
        table_gray = np.zeros((m,m))
        for bvec in range(m):
            table_gray[bvec,np.where(natural_number==bvec)[0]] =1

        return table_gray

def gen_const(bit_per_cu):
    
    points_per_dim = 2**(bit_per_cu // 2)
    
    A = np.arange(0, points_per_dim)
    A = A - np.mean(A)
    
    C = []
    for i in A:
        for j in A:
            C.append([i,j])
    C = np.array(C)
    
    L = gen_labeling(bit_per_cu)
    L = np.argmax(L, axis=1)
    C = C[L]
    
    return C


class GS(nn.Module):
    def __init__(self,  bit_per_cu, init_qam=False):
        super(GS, self).__init__()
        self.bit_per_cu = bit_per_cu
        self.init_qam = init_qam
        if self.init_qam:
            C = gen_const(self.bit_per_cu)
            en_point = np.mean(np.sum(np.square(C), axis=1))
            C = C / np.sqrt(en_point)
            self.Cr = torch.tensor(C[:,0],dtype = torch.float())#,requires_grad=True)
            self.Ci = torch.tensor(C[:,1],dtype = torch.float())#,requires_grad=True)
            self.Cr = nn.Parameter(self.Cr)
            self.Ci = nn.Parameter(self.Ci)
        else:
            const_size = 2**self.bit_per_cu
            self.Cr = torch.normal(mean = 0.0,std=0.05,size = (1,const_size)).squeeze()#.requires_grad_(True)
            self.Ci = torch.normal(mean = 0.0,std=0.05,size = (1,const_size)).squeeze()#.requires_grad_(True)
            self.Cr = nn.Parameter(self.Cr)
            self.Ci = nn.Parameter(self.Ci)
    def get_const(self):
        C = torch.complex(self.Cr, self.Ci)
        # Center the constellation
        C = C - torch.mean(C)
        
        # Normalization
        en_point = torch.mean(torch.square(torch.real(C)) + torch.square(torch.imag(C)))
        norm_fac = torch.complex(torch.sqrt(en_point), torch.tensor(0.0).cuda())
        C = torch.div(C, norm_fac)
        if np.inf in C:
            C[C == np.inf] = torch.tensor(0.0).cuda()
        
        return C
    def forward(self,s):
        
        batch_size = s.shape[0]
        C = self.get_const()
        C = torch.tile(torch.unsqueeze(C, axis=0), (batch_size, 1))
                
        # Mapping
        s = s.long()
        x = torch.gather(C,1,s)
        
        return x


# class SeparableConv2d(torch.nn.Module):
#     def __init__(self, 
#                  in_channels,
#                  out_channels,
#                  kernel_size,
#                  dilation_size,
#                  padding_size,
#                  bias=True
#         ):
#         super(SeparableConv2d,self).__init__()
#         self.spatialConv = convnxn(
#              in_channels=in_channels,
#              out_channels=out_channels,
#              kernel_size=kernel_size,
#              padding_size=padding_size,
#              dilation_size=dilation_size,
#              groups_size=in_channels
#         )
#         self.pointConv = convnxn(
#              in_channels=in_channels,
#              out_channels=out_channels,
#              kernel_size=1,
#              padding_size=0,
#              dilation_size=1,
#              groups_size = 1
#         )
    
#     def forward(self, x, params=None, episode=None):
#         return self.pointConv(self.spatialConv(x, get_child_dict(params, 'spatialConv')), get_child_dict(params, 'pointConv'))


class ResNetBlock(nn.Module):
    
    def __init__(self, filters, kernel_size,dilation_rate, use_dense,padding_size,training):
        super(ResNetBlock, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.dilation_rate = dilation_rate
        self.use_dense = use_dense
        self.padding_size = padding_size
        self.training = training
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        if self.use_dense:
            self.dense = nn.Linear(self.filters,self.filters,bias = False)
        self.bn1 = nn.BatchNorm2d(self.filters, affine = self.training)
        self.conv1 = nn.Conv2d(self.filters,self.filters, self.kernel_size, dilation=self.dilation_rate, padding=self.padding_size,groups = 1)
        self.bn2 = nn.BatchNorm2d(self.filters, affine = self.training)
        self.conv2 = nn.Conv2d(self.filters,self.filters, self.kernel_size, dilation=self.dilation_rate, padding=self.padding_size,groups = 1)
    
    def forward(self, x):
        z = x
        
        z = self.bn1(z)
        z = self.relu1(z)
        z = self.conv1(z)
        z = self.bn2(z)
        z = self.relu2(z)
        z = self.conv2(z)
#         z = self.bn1(z)
#         z = self.relu1(z)
#         z = self.conv1(z)
#         z = self.bn2(z)
#         z = self.relu2(z)
#         z = self.conv2(z)
        if self.use_dense:
            y = self.dense(x)
        else:
            y = x
        z = z + y
        
        return z

    
class OFDM:
    
    # num_subc: number of subcarriers
    # cp_length: Cycle prefix duration [symbols]
    # use_cp: If True, uses a cycling-prefix, otherwise does not.
    def __init__(self, num_subc, cp_length, use_cp=True):
        
        self.num_subc = num_subc
        self.cp_length = cp_length
        self.use_cp = use_cp
    
    # x_f: Baseband symbols in the frequency domain [batch size, number of subcarriers, number of OFDM symbols]
    # The ordering of the subcarriers is assumed to be as follows: negatve frequencies (in order of decreasingly negative frequency),
    #  then center frequency, then positive frequencies.
    def modulate(self, x_f):
        
        # Checking the number of subcarriers is large enough given the CP length
        # tf.debugging.assert_positive(tf.shape(x_f)[1] - self.cp_length, "Number of subcarriers too small given the specified CP length")
        
        x_f = torch.transpose(x_f, 2, 1)
        
        # FFT shift
        x_f = torch.fft.ifftshift(x_f, dim=2)
        
        # Moving to time domain
        x = torch.fft.ifft(x_f)*torch.complex(torch.sqrt(torch.tensor(x_f.shape[-1])), torch.tensor(0.0))
        
        # Adding CP
        if self.use_cp:
            x = torch.cat((x[:,:,-self.cp_length:], x), dim=2)
        
        # Reshaping to get a single time domain signal
        x = x.view(x.shape[0], -1)
        
        return x
        
    
    # y_t: Baseband symbols in the time domain [batch size, time steps]
    def demodulate(self, y_t):
        
        if self.use_cp:
            symbol_length = self.num_subc + self.cp_length
        else:
            symbol_length = self.num_subc
        
        # Checking the signal dimension matches the specified CP length and number of subcarriers
        num_symbs = torch.div(y_t.shape[1], symbol_length)
        rem_symbs = torch.fmod(torch.tensor(y_t.shape[1]), symbol_length)
#         tf.debugging.assert_positive(num_symbs, "Signal duration is too short given specified OFDM configuration")
#         tf.debugging.assert_equal(rem_symbs, 0, "Signal duration does not match specified OFDM configuration")
        
        # Reshaping
        y = y_t.view(y_t.shape[0], -1, symbol_length)
        
        # Removing CP
        if self.use_cp:
            y = y[:,:,self.cp_length:]
        
        # Moving to frequency domain
        y = torch.fft.fft(y)/torch.complex(torch.sqrt(torch.tensor(y.shape[-1])), torch.tensor(0.0))
        
        # FFT shift
        y = torch.fft.fftshift(y, dim=2)
        
        y = torch.transpose(y, 2, 1)
        
        return y
    

class MultiPathChannel(nn.Module):
    
    def __init__(self):
        super(MultiPathChannel, self).__init__()
         
    # x: time domain baseband channel input symbols, with shape [batch_size, number of time steps]
    # h: channel response in time domain, with shape [batch_size, number of time steps, number of taps]
    # noise_power_db: Noise power in dB. Scalar
    def forward(self, x, h, noise_power_db):
        
        batch_size = x.shape[0]
        time_steps_no = x.shape[1]
        taps_no = h.shape[2]
        
        # If extra time steps are available for the channel, we just ignore them
        h = h[:,:time_steps_no,:]
        # Reshaping the channel input symbols
        x = torch.unsqueeze(x, dim=2)
        x = torch.tile(x, (1, 1, taps_no))
        padding_r = torch.zeros((batch_size, taps_no, taps_no))
        padding_i = torch.zeros((batch_size, taps_no, taps_no))
        padding = torch.complex(padding_r,padding_i)
        padding = padding
        x = torch.cat((x, padding.cuda()), axis=1)
        x_ = torch.transpose(torch.zeros_like(torch.transpose(torch.zeros_like(x), 0,2)), 1,2)
        for i in range(taps_no):
            z = x[:,:,i]
            z = torch.roll(z, i, 1)
            x_[i] = z
        x_ = torch.transpose(x_, 0,1)
        x_ = torch.transpose(x_, 1,2)
        x = x_[:,:time_steps_no,:] # [batch size, number of timer steps, number of taps]
        # Channel transfer function
        y = torch.sum(h*x, axis=2)
        noise_std = torch.sqrt(0.5*(torch.pow(10.0, noise_power_db/10.0)))
        noise_std = torch.unsqueeze(noise_std, dim=1)
        noise_std = noise_std.cpu().numpy()
        noise = (np.random.normal(0.0, noise_std,size = (y.shape[0],y.shape[1])), np.random.normal(0.0, noise_std,size = (y.shape[0],y.shape[1])))
        noise = torch.complex(torch.tensor(noise[0]), torch.tensor(noise[1]))
        noise = noise.cuda()
        y = y + noise
        y = y.to(dtype=torch.complex64)
        return y

    
class RXNN(nn.Module):
    
    def __init__(self, bit_per_cu,training):
        super(RXNN, self).__init__()
        
        self.bit_per_cu = bit_per_cu
        self.training = training
        self.conv_in = nn.Conv2d(5,256, (3,3), dilation=(1,1), padding=(1,1))#padding 1,1
        self.res1 = ResNetBlock(256, (3,3), (3,1),False,(3,1),training)#padding 3,1
        self.res2 = ResNetBlock(256, (3,3), (6,2), False,(6,2),training)#padding 6,2
        self.res3 = ResNetBlock(256, (3,3), (6,2), False,(6,2),training)#padding 6,2
        self.res4 = ResNetBlock(256, (3,3), (3,1), False,(3,1),training)#padding 3,1
        self.conv_out = nn.Conv2d(256,4, (1,1), dilation=(1,1), padding=(0,0))#padding 0,0
    
    # all inputs of shape [nm subframes, num subcarriers, block length]
    # y: received signal
    # snr_db: SNR [dB]
    def forward(self, y, snr_db):
        
        # Positional encoding
        num_frames = y.shape[0]
        num_subc = y.shape[1]
        block_length = y.shape[2]
        spectral_pos_enc = torch.linspace(0.0, 1.0, num_subc)
        time_pos_enc = torch.linspace(0.0, 1.0, block_length)
        spectral_pos_enc = torch.unsqueeze(spectral_pos_enc, dim =1)
        spectral_pos_enc = torch.tile(spectral_pos_enc, (1, block_length))
        time_pos_enc = torch.unsqueeze(time_pos_enc, axis=0)
        time_pos_enc = torch.tile(time_pos_enc, (num_subc, 1))
        pos_enc = torch.stack([spectral_pos_enc, time_pos_enc], dim =2)
        pos_enc = torch.unsqueeze(pos_enc, axis=0)
        pos_enc = torch.tile(pos_enc, (num_frames, 1, 1, 1))

        #         # Inputs
        snr_db = snr_db
        pos_enc = pos_enc.cuda()
        z = torch.stack((torch.real(y), torch.imag(y),snr_db),dim = 3)
        z = torch.cat((z, pos_enc), dim=3)
        z = torch.transpose(z, 1, 3)
        z = torch.transpose(z, 2, 3)
        # Call conv layers
#         print(z)
        z = z.float()
        z = self.conv_in(z)
        z = self.res1(z)
        z = self.res2(z)
        z = self.res3(z)
        z = self.res4(z)
        z = self.conv_out(z)
        z = torch.transpose(z, 1, 3)
        z = torch.transpose(z, 1, 2)
        return z


class SysGS_torch_gs(nn.Module):
    def __init__(self):
        super(SysGS_torch_gs, self).__init__()
        
        self.gs = GS(4)
        self.ofdm = OFDM(72, 6, False)
        self.channel = MultiPathChannel()
        self.rx = RXNN(4,True)
    def forward(self,snr_db,h,s,params=None, episode=None):
        noise_power_db = -snr_db
#         print(s.shape)
        x = self.gs(s)
        x = x.view(10,72,14)
        x = self.ofdm.modulate(x)
        y = self.channel(x,h,noise_power_db)
        y = self.ofdm.demodulate(y)
        snr_db_rx = torch.tile(torch.unsqueeze(torch.unsqueeze(snr_db, dim=1), dim=2), [1, 72, 14])
        y = self.rx(y,snr_db_rx)
        return y

In [14]:
model = SysGS_torch_gs()

In [37]:
model.load_state_dict(path)

<All keys matched successfully>

In [38]:
class dataset(Dataset):

    def __init__(self,input_path, target_path, mode = 'train', seedNN = 0):
        super(dataset, self).__init__()
        self.input_data = []
        self.target_data = []
        input_data_meta = []
        target_data_meta = []
        self.mode_t = mode
        for i in input_path:
            input_data_meta += list(np.load(i))
        for i in target_path:
            target_data_meta += list(np.load(i))
        input_data_meta = np.array(input_data_meta)
        target_data_meta = np.array(target_data_meta)
        self.length = input_data_meta.shape[0]
        random.seed(seedNN)
        ind_shuffle = random.sample(range(0, self.length), self.length)
        if mode == 'train':
            self.target_data = target_data_meta[:int(0.8 * self.length)]
            self.input_data = input_data_meta[:int(0.8 * self.length)]
#             self.input_data = np.reshape(self.input_data,(int(self.input_data.shape[0]/10),10,self.input_data.shape[1],self.input_data.shape[2]))
#             self.target_data = np.reshape(self.target_data,(int(self.target_data.shape[0]/10),10,self.target_data.shape[1],self.target_data.shape[2]))
        else:
            self.target_data = target_data_meta[int(0.8 * self.length):]
            self.input_data = input_data_meta[int(0.8 * self.length):]
#             self.input_data = np.reshape(self.input_data,(int(self.input_data.shape[0]/10),10,self.input_data.shape[1],self.input_data.shape[2]))
#             self.target_data = np.reshape(self.target_data,(int(self.target_data.shape[0]/10),10,self.target_data.shape[1],self.target_data.shape[2]))
        self.length = self.input_data.shape[0]
    def __len__(self):
#         if self.mode_t:
#             return 10
#         else:
#             return 10
        return 10

    def __getitem__(self, index):
#         print(self.input_data[index].shape)
        input_data = torch.from_numpy(self.input_data[index])
        target_data = torch.from_numpy(self.target_data[index])
        return input_data, target_data

In [39]:
trainfolder = ['/datadisk/sdc/wangruibo/maml_e2e/140-200-low/dataset_UMa_D=140.0-200.0_&_A=0.0-0.66_s=10000.npy',
            '/datadisk/sdc/wangruibo/maml_e2e/140-200-medium/dataset_UMa_D=140.0-200.0_&_A=0.66-1.33_s=10000.npy',
            '/datadisk/sdc/wangruibo/maml_e2e/140-200-high/dataset_UMa_D=140.0-200.0_&_A=1.33-2.0_s=10000.npy']
testfolder=['/datadisk/sdc/wangruibo/maml_e2e/bit_labels_s_size=10000.npy',
           '/datadisk/sdc/wangruibo/maml_e2e/bit_labels_s_size=10000.npy',
           '/datadisk/sdc/wangruibo/maml_e2e/bit_labels_s_size=10000.npy']

In [40]:
Umadata_train = dataset(trainfolder,testfolder,mode = 'train')
Umadata_test = dataset(trainfolder,testfolder,mode = 'val')

In [41]:
# trainloader = DataLoader(Umadata_train, batch_size=10, shuffle=False, num_workers=8)
# testloader = DataLoader(Umadata_test, batch_size=10, shuffle=False, num_workers=8)
trainloader = DataLoader(Umadata_train, batch_size=10, shuffle=False, num_workers=8)
testloader = DataLoader(Umadata_test, batch_size=10, shuffle=False, num_workers=8)

In [42]:
import torch.optim as optim
def e2e_traninig(net,Epoch,trainloader, testloader,SaveFolder,savename,maml = True,evaluation=False):
    
    # Construct NN and parameters
#     net = SubCAN(32, 3, 6, 8).cuda()
    NetworkParamNum = "Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in net.parameters()))
    print(NetworkParamNum)
    
    # Construct scheduler
    lr = 0.00001
    # criterion = nn.L1Loss()
    # optimizer = optim.SGD(net.parameters(), lr=lr, weight_decay=10**(-5))
    optimizer = optim.Adam(net.parameters(), lr=lr)
#     scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
    
    # Initialize some parameters
    min_testloss = 100
    testError  = []
    trainError = []
    costTime   = []
    BER_train = []
    BER_test = []
    KL_train = []
    KL_test = []
    storeStepResult = True
    minloss = 1000000000
    snr_db = torch.DoubleTensor(1, 10).uniform_(0.0, 15.0).squeeze()
    zero_loss_train = 0
    zero_loss_eval = 0
    zero_ber_trian = 0
    zero_ber_test = 0
    for dmrs, target in trainloader:
        net.eval()
        target = target.cuda()
        dmrs = dmrs.cuda()
        A = np.flip([[[2**i for i in range(4)]]])
        s = torch.sum(target.cpu()*A, axis=2)
        snr_db = snr_db.cuda()
        s = s.cuda()
        t_pred = net(snr_db,dmrs,s)
        LLR = t_pred
        LLR = torch.where(LLR >= 0, 1,-1)
        LLR = (LLR+1)/2
        LLR = LLR.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
        zero_ber_trian += (1 - ((LLR==target).sum().item()/(t_pred.shape[0]*t_pred.shape[1]*t_pred.shape[2]*t_pred.shape[3])))
        t_pred = t_pred.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
        loss = F.binary_cross_entropy_with_logits(t_pred,target.float())
        
        zero_loss_train += loss.item()
    BER_train.append(zero_ber_trian/len(trainloader))
    trainError.append(zero_loss_train/len(trainloader))
    if evaluation:
        for dmrs, target in testloader:
            net.eval()
            target = target.cuda()
            dmrs = dmrs.cuda()
            A = np.flip([[[2**i for i in range(4)]]])
            s = torch.sum(target.cpu()*A, axis=2)
            snr_db = snr_db.cuda()
            s = s.cuda()
            t_pred = net(snr_db,dmrs,s)
            LLR = t_pred
            LLR = torch.where(LLR >= 0, 1,-1)
            LLR = (LLR+1)/2
            LLR = LLR.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
            zero_ber_test += (1 - ((LLR==target).sum().item()/(t_pred.shape[0]*t_pred.shape[1]*t_pred.shape[2]*t_pred.shape[3])))
            t_pred = t_pred.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
            loss = F.binary_cross_entropy_with_logits(t_pred,target.float())
            zero_loss_eval += loss.item()
        BER_test.append(zero_ber_test/len(testloader))
        testError.append(zero_loss_eval/len(testloader))
    for epoch in range(Epoch):
        
#         randSeed = random.randint(0,10000)
#         if len(ChannelType) == 1:
#             trainloader, testloader = DataSplitter(ChannelType, MaxDopplerList, SNRList, seedNN = randSeed)
#         elif len(ChannelType) > 1:
#             trainloader, testloader = DataSplitterMix(ChannelType, MaxDopplerList, SNRList, seedNN = randSeed) 
        net.train()
        train_loss = 0.0
        test_loss  = 0.0      
        numTrain = 0
        numTest  = 0 
        bertrain = 0
        bertest = 0
        time_start = time.time()
        for dmrs, target in trainloader:
            optimizer.zero_grad()
            target = target.cuda()
            dmrs = dmrs.cuda()
#             logit = dmrs.view(dmrs.shape[0]*dmrs.shape[1],dmrs.shape[2],dmrs.shape[3],dmrs.shape[4])
#             label = target.view(target.shape[0]*target.shape[1],target.shape[2],target.shape[3],target.shape[4])
            A = np.flip([[[2**i for i in range(4)]]])
            s = torch.sum(target.cpu()*A, axis=2)
            snr_db = snr_db.cuda()
            s = s.cuda()

            t_pred = net(snr_db,dmrs,s)
            LLR = t_pred
            LLR = torch.where(LLR >= 0, 1,-1)
            LLR = (LLR+1)/2
            LLR = LLR.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
            bertrain += (1 - ((LLR==target).sum().item()/(t_pred.shape[0]*t_pred.shape[1]*t_pred.shape[2]*t_pred.shape[3])))
    #             print(t_pred.shape)
            t_pred = t_pred.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
            loss = F.binary_cross_entropy_with_logits(t_pred,target.float())
                    # loss = criterion(t_pred, target)
            loss.backward()
            optimizer.step()
            train_loss += float(loss.item())
#         scheduler.step()
            
        net.eval()
        numTrain += len(trainloader)
            
        del t_pred, target, dmrs        
            # 测试
        if evaluation:
            for dmrs, target in testloader:
                target = target.cuda().float()
                dmrs = dmrs.cuda().float()
    #             logit = dmrs.view(dmrs.shape[0]*dmrs.shape[1],dmrs.shape[2],dmrs.shape[3],dmrs.shape[4])
    #             label = target.view(target.shape[0]*target.shape[1],target.shape[2],target.shape[3],target.shape[4])
                A = np.flip([[[2**i for i in range(4)]]])
                s = torch.sum(target.cpu()*A, axis=2)
                snr_db = snr_db.cuda()
                s = s.cuda()

                t_pred = net(snr_db,dmrs,s)
                LLR = t_pred
                LLR = torch.where(LLR >= 0, 1,-1)
                LLR = (LLR+1)/2
                LLR = LLR.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
                bertest += (1 - ((LLR==target).sum().item()/(t_pred.shape[0]*t_pred.shape[1]*t_pred.shape[2]*t_pred.shape[3])))
                t_pred = t_pred.view(t_pred.shape[0],t_pred.shape[1]*t_pred.shape[2],t_pred.shape[3])
        #             print(t_pred.shape)
                loss = F.binary_cross_entropy_with_logits(t_pred,target.float())
#                 KL = KL(t_pred,target)
                        # loss = criterion(t_pred, target)
                test_loss += float(loss.item())
            numTest += len(testloader)
            del t_pred, target, dmrs
        
        time_end = time.time()
        
        train_loss /= numTrain
        bertrain /= numTrain
#         train_loss /= torch.log(torch.tensor([2.]))
#         train_loss = torch.mean(torch.tensor(4.)-torch.sum(train_loss,axis = 3))
        if evaluation:
            test_loss  /= numTest
            bertest /= numTest
            
#         test_loss /= torch.log(torch.tensor([2.]))
        # For print and record Loss
        cost = time_end - time_start
        if evaluation:
            showStrings = '[Epoch: %3d/%3d][TraingLoss: %3.5f][TestingLoss:%3.5f][Ber train:%3.5f][Ber test:%3.5f][CostTime: %3.5f s]' % (epoch+1, Epoch, train_loss, test_loss,bertrain,bertest, cost)
        else:
            showStrings = '[Epoch: %3d/%3d][TraingLoss: %3.5f][Ber:%3.5f][CostTime: %3.5f s]' % (epoch+1, Epoch, train_loss, bertrain, cost)
        print(showStrings, end='\n')
        if maml:
            name = 'MAML' + savename
        else:
            name = 'Joint' + savename
        trainError.append(train_loss)
        testError.append(test_loss)
        costTime.append(cost)
        BER_train.append(bertrain)
        BER_test.append(bertest)
        file = open(SaveFolder + name + '.txt' , 'w')
        file.write(NetworkParamNum + "\n")
        file.write('TrainingLoss = ' + str(trainError) + "\n")
        file.write('TestingLoss = '  + str(testError) + "\n")
        file.write('CostTime Epoch (s): ' + str(costTime) + "\n")
        file.write('BER train: ' + str(BER_train) + "\n")
        file.write('BER test: ' + str(BER_test) + "\n")
        file.close()

In [43]:
txt_name = '_only_far_on_far_500'
e2e_traninig(model.cuda(),20,trainloader,testloader,'/datadisk/sdc/wangruibo/Code_MAML_newframework_e2e/PyTorch-MAML-master/E2E_MAML_compare/',txt_name)

Total number of paramerters in networks is 4737572  
[Epoch:   1/ 20][TraingLoss: 0.71671][Ber:0.49913][CostTime: 4.98355 s]
[Epoch:   2/ 20][TraingLoss: 0.71135][Ber:0.49439][CostTime: 4.32723 s]
[Epoch:   3/ 20][TraingLoss: 0.71020][Ber:0.49157][CostTime: 4.85246 s]
[Epoch:   4/ 20][TraingLoss: 0.70938][Ber:0.49000][CostTime: 4.88562 s]
[Epoch:   5/ 20][TraingLoss: 0.70733][Ber:0.48710][CostTime: 4.82967 s]
[Epoch:   6/ 20][TraingLoss: 0.70506][Ber:0.48380][CostTime: 4.71952 s]
[Epoch:   7/ 20][TraingLoss: 0.70379][Ber:0.48058][CostTime: 4.97334 s]
[Epoch:   8/ 20][TraingLoss: 0.70095][Ber:0.47788][CostTime: 4.86455 s]
[Epoch:   9/ 20][TraingLoss: 0.69983][Ber:0.47431][CostTime: 4.93124 s]
[Epoch:  10/ 20][TraingLoss: 0.69854][Ber:0.47316][CostTime: 4.84218 s]
[Epoch:  11/ 20][TraingLoss: 0.69760][Ber:0.47029][CostTime: 4.72361 s]
[Epoch:  12/ 20][TraingLoss: 0.69616][Ber:0.46682][CostTime: 4.70823 s]
[Epoch:  13/ 20][TraingLoss: 0.69464][Ber:0.46292][CostTime: 4.77929 s]
[Epoch:  14